In [1]:
import os
import pandas as pd
import openpyxl

# Define the directory where your CSV files are located
csv_directory = 'csvs'  # Change this to the directory containing your CSV files

# Define the file to exclude
excluded_file = 'Sets.csv'

# Define the directory where you want to save the filtered CSV files
output_csv_directory = 'output_csv'  # Change this to the directory where you want to save the filtered CSV files

# Create the output directory for CSV files if it doesn't exist
os.makedirs(output_csv_directory, exist_ok=True)

# Define the directory where you want to save the Excel file
output_excel_directory = 'output_excel'  # Change this to the directory where you want to save the Excel file

# Create the output directory for the Excel file if it doesn't exist
os.makedirs(output_excel_directory, exist_ok=True)

# Specify the Excel file path
excel_file_path = 'GENeSYS-MOD_User_Input_Settings_v03_phe_06-09-2023.xlsx'  # Replace with the path to your Excel file


# Create a list of unique values in the fourth column of the filtered DataFrame
#unique_values_region = filtered_df_region.iloc[:, 3].unique()  # Assuming the fourth column is indexed at 3 (0-based index)
#unique_values_technology = filtered_df_technology.iloc[:, 0].unique()  # Assuming the first column is indexed at 0 (0-based index)
#unique_values_storage = filtered_df_storage.iloc[:, 0].unique()  # Assuming the first column is indexed at 0 (0-based index)
#unique_values_year = filtered_df_year.iloc[:, 0].unique()  # Assuming the first column is indexed at 0 (0-based index)


In [2]:
# Create a list of sheet names to read
sheets_to_read = ['Region_selection', 'Technology_selection', 'Storage_selection', 'Fuel_selection', 'Mode_of_operation_selection', 'Emission_selection', 'Modal_type_selection', 'Sector_selection', 'Year_selection']

# Initialize an empty dictionary to store DataFrames
data_frames = {}
filtered_df = {}
unique_values = {}

unique_values_concatenated = pd.DataFrame()


# Read sheets and store them in the dictionary
for sheet_name in sheets_to_read:
    data_frames = pd.read_excel(excel_file_path, engine='openpyxl', sheet_name=sheet_name)

    filtered_df= data_frames[data_frames.iloc[:, 1] == 1]# Assuming the second column is indexed at 1 (0-based index)

    unique_values= pd.DataFrame(filtered_df.iloc[:, 0].unique())  # Assuming the first column is indexed at 0 (0-based index)
    unique_values_parameter = pd.DataFrame(unique_values)
    
    unique_values_concatenated = pd.concat([unique_values_concatenated, unique_values], axis=1)
    
# Create a CSV file containing unique values
unique_values_csv_file_path = os.path.join(output_csv_directory, 'Sets.csv')
unique_values_concatenated.to_csv(unique_values_csv_file_path, index=False, decimal='.') 

In [ ]:
# Create DataFrames for unique values
unique_values_region_df = pd.DataFrame({'Region': unique_values_region})
unique_values_technology_df = pd.DataFrame({'Technology': unique_values_technology})
unique_values_storage_df = pd.DataFrame({'Storage': unique_values_storage})
unique_values_year_df = pd.DataFrame({'Year': unique_values_year})

# Concatenate the DataFrames vertically
unique_values_concatenated = pd.concat([unique_values_region_df, unique_values_technology_df, unique_values_year_df, unique_values_storage_df], axis=1)

# Create a CSV file containing unique values
unique_values_csv_file_path = os.path.join(output_csv_directory, 'Sets.csv')
unique_values_concatenated.to_csv(unique_values_csv_file_path, index=False, decimal='.')

In [ ]:
# Initialize an empty dictionary to store DataFrames
data_frames = {}


In [ ]:
# Initialize the Excel writer
output_excel_file_path = os.path.join(output_excel_directory, 'output.xlsx')
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    unique_values_concatenated.to_excel(writer, sheet_name='Sets', index=False, header=True)
    for filename in os.listdir(csv_directory):
        if filename.endswith('.csv') and filename != excluded_file:
            # Construct the full path to the CSV file
            csv_file_path = os.path.join(csv_directory, filename)
            
            df_first = pd.read_csv(csv_file_path, delimiter=';',header=None)
            df_header = df_first.iloc[:4] #shweta
            df_header = df_header.apply(lambda x: x.apply(lambda y: str(y).replace('.0', '')))
            df_header = df_header.iloc[:4, :1]
            
            # Read the CSV file into a Pandas DataFrame, skipping the first 4 rows
            df = pd.read_csv(csv_file_path, delimiter=';', skiprows=4)
            
            if "Region" in df.columns:
                # Filter the DataFrame based on whether the values in the "Region" column are present in unique_values
                filtered_df = df[df["Region"].isin(unique_values_region)]
                if "Region2" in df.columns:
                    filtered_df = filtered_df[filtered_df["Region2"].isin(unique_values_region)]  
            else:
                # If "Region" column is not found, copy the entire DataFrame
                filtered_df = df.copy()
                
            if "Technology" in df.columns:
                filtered_df = filtered_df[filtered_df["Technology"].isin(unique_values_technology)]
            else:
                filtered_df = filtered_df.copy()
                
            if "Year" in df.columns:
                filtered_df = filtered_df[filtered_df["Year"].isin(unique_values_year)]
            else:
                filtered_df = filtered_df.copy()    
                
            filtered_df = pd.concat([filtered_df.columns.to_frame().T, filtered_df], ignore_index=True)
            filtered_df.columns = range(len(df.columns))
            
            df_final = pd.concat([df_header, filtered_df], axis=0)
            
            df_final.replace('nan', '', inplace=True)
            df_final.apply(lambda x: x.apply(lambda y: str(y).replace('.', ',')))
            
            # Get the worksheet name without the .csv extension
            worksheet_name = os.path.splitext(filename)[0]
            
            # Store the DataFrame in the dictionary with the filename (without extension) as the key
            data_frames[worksheet_name] = df_final
            
            # Write the DataFrame to an Excel worksheet with the same filename (without extension)
            df_final.to_excel(writer, sheet_name=worksheet_name, index=False, header=False)
            
            # Construct the full path to the output CSV file
            output_csv_file_path = os.path.join(output_csv_directory, filename)
            
            # Save the filtered DataFrame to the CSV file without the index
            df_final.to_csv(output_csv_file_path, index=False, header=False, decimal='.')